In [142]:
import pandas as pd
import geopandas as gpd
import numpy as np
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import *
import psycopg2
import time
%matplotlib inline

pd.options.display.max_rows = 999
pd.options.display.max_columns = None

In [143]:
password = open('password.txt','r').readline()
username = 'kmerrem'

#this is to write from geopandas to postgis table 
#gisdata_engine = create_engine('postgresql://%s:%s@172.19.118.205:5432/gisdata' %(username,password))
walkability_engine = create_engine('postgresql://%s:%s@172.19.118.205:5432/walkability' %(username,password))

In [144]:
#buildings = gpd.read_postgis("select * from staging.buildings322", walkability_engine, 'geom')
buildings = gpd.read_postgis("select * from staging.buildings327", walkability_engine, 'geom')
cts = gpd.read_postgis("SELECT * FROM public.census_tracts_blank", walkability_engine, 'geom')
#pednet = gpd.read_postgis("select * from compliance.att_pednet_all_in WHERE geom IS NOT NULL", walkability_engine, 'geom')
pednet = gpd.read_postgis("select * from staging.a_z1_pednet WHERE geom IS NOT NULL", walkability_engine, 'geom')

In [145]:
buildings.head(2)

,id,geom,field_1,ctuid,ctname,ctnum,geotext,node_ids,d_fc_0,d_ff_0,d_sm_0,d_hp_0,d_sxhc_0,d_wc_0,d_sc_0,d_ar_0,d_dc_0,d_di_0,d_ems_0,d_fc_1,d_ff_1,d_sm_1,d_hp_1,d_sxhc_1,d_wc_1,d_sc_1,d_ar_1,d_dc_1,d_di_1,d_ems_1,d_frc_0,d_fr_0,d_lb_0,d_plc_0,d_rec_0,d_thing_0,d_vtl_0,d_bp_0,d_bs_0,d_ttcstop_0,d_ttcst_0,d_ttcacc_0,d_ts_0,d_frc_1,d_fr_1,d_lb_1,d_plc_1,d_rec_1,d_thing_1,d_vtl_1,d_bp_1,d_bs_1,d_ttcstop_1,d_ttcst_1,d_ttcacc_1,d_ts_1,m_fc_0,m_fc_1,m_ff_0,m_ff_1,m_sm_0,m_sm_1,m_hp_0,m_hp_1,m_sxhc_0,m_sxhc_1,m_wc_0,m_wc_1,m_sc_0,m_sc_1,m_ar_0,m_ar_1,m_dc_0,m_dc_1,m_di_0,m_di_1,m_ems_0,m_ems_1,m_frc_0,m_frc_1,m_fr_0,m_fr_1,m_lb_0,m_lb_1,m_plc_0,m_plc_1,m_rec_0,m_rec_1,m_thing_0,m_thing_1,m_vtl_0,m_vtl_1,m_bp_0,m_bp_1,m_bs_0,m_bs_1,m_ttcstop_0,m_ttcstop_1,m_ttcst_0,m_ttcst_1,m_ttcacc_0,m_ttcacc_1,m_ts_0,m_ts_1,n_fc_0n,n_fc_0type,n_ff_0n,n_ff_0type,n_sm_0n,n_sm_0type,n_hp_0n,n_hp_0type,n_sxhc_0n,n_sxhc_0type,n_wc_0n,n_wc_0type,n_sc_0n,n_sc_0type,n_ar_0n,n_ar_0type,n_lb_0n,n_lb_0type,n_dc_0n,n_dc_0type,n_fr_0n,n_fr_0type,n_di_0n,n_di_0type,n_ems_0n,n_ems_0type,n_frc_0n,n_frc_0type,n_plc_0n,n_plc_0type,n_rec_0n,n_rec_0type,n_thing_0n,n_thing_0type,n_vtl_0n,n_vtl_0type,n_bp_0type,n_bs_0type,n_ttcstop_0n,n_ttcstop_0type,n_ttcst_0n,n_ttcst_0type,n_ttcacc_0n,n_ttcacc_0type,n_ts_0n,n_ts_0type,n_fc_1n,n_fc_1type,n_ff_1n,n_ff_1type,n_sm_1n,n_sm_1type,n_hp_1n,n_hp_1type,n_sxhc_1n,n_sxhc_1type,n_wc_1n,n_wc_1type,n_sc_1n,n_sc_1type,n_ar_1n,n_ar_1type,n_lb_1n,n_lb_1type,n_dc_1n,n_dc_1type,n_fr_1n,n_fr_1type,n_di_1n,n_di_1type,n_ems_1n,n_ems_1type,n_frc_1n,n_frc_1type,n_plc_1n,n_plc_1type,n_rec_1n,n_rec_1type,n_thing_1n,n_thing_1type,n_vtl_1n,n_vtl_1type,n_bp_1type,n_bs_1type,n_ttcstop_1n,n_ttcstop_1type,n_ttcst_1n,n_ttcst_1type,n_ttcacc_1n,n_ttcacc_1type,n_ts_1n,n_ts_1type
0,1,(POLYGON ((303785.8508000001 4841002.746400001...,0,5350282.0,282.0,535028200.0,535028200,89416,487.143005,3179.489990,487.143005,3395.404053,2207.837891,2177.249023,60.861000,2361.337891,927.267029,1765.776001,1580.375977,570.526978,3179.489990,1656.931030,3606.419922,5000.0,3683.527100,696.325012,2678.096924,1544.262939,2385.887939,4537.008789,1624.584961,1580.375977,1708.963989,1535.342041,934.023987,3662.687988,696.325012,3376.153076,5000.0,175.798004,5000.000000,5000.000000,767.481018,1764.870972,2391.837891,2385.887939,1535.342041,1014.937012,4353.133789,850.755005,5000.000000,5000.0,198.593002,5000.000000,5000.000000,961.98999,6.765875,7.923986,44.159583,44.159583,6.765875,23.012931,47.158390,50.089166,30.664415,69.444444,30.239570,51.160099,0.845292,9.671181,32.796360,37.195791,12.878709,21.448096,24.524667,33.137332,21.949666,63.014011,22.563680,24.512097,21.949666,33.219971,23.735611,33.137332,21.324195,21.324195,12.972555,14.096347,50.870667,60.460192,9.671181,11.816042,46.891015,69.444444,69.444444,69.444444,23.735611,33.137332,69.444444,69.444444,69.444444,69.444444,10.659459,13.360972,Shop N Milk,cs,NEW PASSIONS RESTAURANT,ff,GRACE FOOD MARTS,sm,Trillium Health Centre - Queensway Health Centre,h,Birth Control and Sexual Health Centre,Eglinton-Lawrence (8),Downsview Medical Centre,w,TERRAVIEW WILLOWFIELD PUBLIC SCHOOL,EP,Cummer Park,North York,Weston Public Library,2 King St,Portage Trail B & A School Program,Portage Trail Junior Elementary School - Build...,FIRE STATION 441,947 MARTIN GROVE RD,Syme-Woolner Drop-In,Syme-Woolner Neighbourhood & Family Centre,EMS 19 Post,Ambulance Station 19,Falstaff Community Centre (OEYC),York South-Weston OEY - Macaulay Child Develop...,Marine Headquarters,259 Queens Quay West,Mount Dennis C. Hall,Recreation Centres,Humber Valley Golf Course,Golf Courses,Pelmo Park,Pellatt Ave,Bike Rack,None,GUILDWOOD PKWY AT PRINCE PHILLIP BLVD,2686,None,None,None,None,JANE ST,TRETHEWEY DR,Pelmo Mini Mart,cs,GUSTO PIZZA,ff,OYINGBO AFRICAN SUPERMARKET,sm,Centre For Addiction And Mental Health - Queen...,h,None,None,Patient Networks - Family Medicine and Health ...,w,PLEASANT PUBLIC SCHOOL,EP,Albion Arena,Etobicoke,None,1700 Wilson Ave,Galloway Ea

In [146]:
#jspatial joining of pednet to get ctnum on pednet segments
pednet_ct = gpd.sjoin(pednet,cts)


In [147]:
pednet_ct.head(2)

,gid_old,__gid_left,edge_id,start_node,end_node,next_left_,abs_next_l,next_right,abs_next_r,left_face,right_face,gid_centr,centreline,linear_nam,linear_n_1,linear_n_2,linear_n_3,linear_n_4,address_l,address_r,parity_l,parity_r,lo_num_l,lo_num_r,hi_num_l,hi_num_r,from_inter,to_interse,oneway_dir,oneway_d_1,feature_co,feature__1,pt2,shortest_d,load_date,load_user,data_autho,obs_pts,road_type,sdwlk_desc,sdwlk_code,gcc_sdwlk_,gcc_sdwl_1,topo_sdwlk,qa_sdwlk_c,crosswalk,geom,last_change_timestamp,last_change_user,last_change_ipaddr,last_used_application,qa_sdwlk_code,qa_sdwlk_code1,crosswalk_boolean,qa_sdwlk_code2,gid_left,zid,f_cwalk_v1,f_cwalk,px,px_type,cwalk_type,index_right,gid_right,__gid_right,ctuid,ctname,pruid,prname,cmauid,cmapuid,cmaname,cmatype,shape_leng,shape_area,ctnum,geotext
0,703,37494.0,10000.0,8896.0,8894.0,30550.0,30550.0,-6384.0,6384.0,15795.0,676.0,68336.0,1146460.0,3803.0,Lake Shore Blvd E,Lake Shore Boulevard East,Lake Shore,Blvd,None,25-25,N,O,0.0,25.0,0.0,25.0,13467661.0,13467576.0,0.0,Against-Digitization,201200.0,Major Arterial,SRID=2019;POINT(314818.420913779 4833584.71111...,0.0,2019-01-15,gepure,gepure,None,Major Arterial,No sidewalk on either side,3.0,0.0,None,0.0,None,None,"(LINESTRING (314815.8607 4833592.2401, 314821....",2019-02-20 17:43:11.671090,postgres,127.0.0.1/32,,None,Pass,True,None,34488,30665,None,Y,211,ts,ts,280,281,265,5350013.02,0013.02,35,Ontario,535,35535,Toronto,B,3525.22685,417800.246307,535001302.0,535001302
13,40429,37459.0,6385.0,8895.0,8893.0,6383.0,6383.0,-64927.0,64927.0,676.0,15789.0,34443.0,1146489.0,1962.0,Lake Shore Blvd W,Lake Shore Boulevard West,Lake Shore,Blvd,None,None,N,N,0.0,0.0,0.0,0.0,13467661.0,13467693.0,1.0,Follow-Digitization,201200.0,Major Arterial,SRID=2019;POINT(314794.688721227 4833568.09292...,0.0,2019-01-15,gepure,gepure,None,Major Arterial,Sidewalk on both sides,7.0,NaN,None,NaN,None,None,"(LINESTRING (314792.3111 4833576.0342, 314797....",2019-02-20 17:43:11.671090,postgres,127.0.0.1/32,,None,Pass,True,None,22847,18932,None,Y,211,ts,ts,280,281,265,5350013.02,0013.02,35,Ontario,535,35535,Toronto,B,3525.22685,417800.246307,535001302.0,535001302


In [148]:
#need to keep left join gid
pednet_sum = pednet_ct[['gid_left','ctuid','ctnum','geom',]]
pednet_sum.head(2)

,gid_left,ctuid,ctnum,geom
0,34488,5350013.02,535001302.0,"(LINESTRING (314815.8607 4833592.2401, 314821...."
13,22847,5350013.02,535001302.0,"(LINESTRING (314792.3111 4833576.0342, 314797...."


In [149]:
summary_buildings  = buildings[['id','ctuid','ctnum','geom']]
summary_buildings.head(2)

,id,ctuid,ctnum,geom
0,1,5350282.0,535028200.0,(POLYGON ((303785.8508000001 4841002.746400001...
1,2,5350265.0,535026500.0,"(POLYGON ((314413.1683999998 4842108.4136, 314..."


In [150]:
#NEED THE INDEX TO RUN
cts =cts.set_index('ctnum')
#summary_buildings = summary_buildings.set_index('ctnum')

In [151]:
cts

,gid,__gid,ctuid,ctname,pruid,prname,cmauid,cmapuid,cmaname,cmatype,shape_leng,shape_area,geotext,geom
ctnum,,,,,,,,,,,,,,
535000100.0,1,62,5350001.00,0001.00,35,Ontario,535,35535,Toronto,B,36906.495474,6.619690e+06,535000100,(POLYGON ((320588.9306980086 4835855.647789237...
535000200.0,2,63,5350002.00,0002.00,35,Ontario,535,35535,Toronto,B,32534.307654,3.293422e+06,535000200,"(POLYGON ((316642.831678931 4832470.554474635,..."
535026400.0,3,87,5350264.00,0264.00,35,Ontario,535,35535,Toronto,B,12241.116870,3.464007e+06,535026400,(POLYGON ((314351.8668838083 4845056.100542194...
535008200.0,4,94,5350082.00,0082.00,35,Ontario,535,35535,Toronto,B,2310.602463,3.082930e+05,535008200,(POLYGON ((318379.9191213865 4837764.438619476...
535000600.0,5,105,5350006.00,0006.00,35,Ontario,535,35535,Toronto,B,455.941409,1.287364e+04,535000600,"(POLYGON ((310234.199135254 4832619.302139398,..."
535016300.0,6,217,5350163.00,0163.00,35,Ontario,535,35535,Toronto,B,2842.202163,4.540284e+05,535016300,(POLYGON ((309832.5952238032 4837976.043997317...
535021302.0,7,443,5350213.02,0213.02,35,Ontario,535,35535,Toronto,B,14621.600658,7.041416e+06,535021302,(POLYGON ((302360.8525544122 4831898.581168552...
535037604.0,8,455,5350376.04,0376.04,35,Ontario,535,35535,Toronto,B,4439.479932,1.193620e+06,535037604,(POLYGON ((318932.6039103945 4852366.218676397...
535024701.0,9,501,5350247.01,0247.01,35,Ontario,535,35535,Toronto,B,23043.297794,1.734167e+07,535024701,"(POLYGON ((294567.331474375 4843841.86104348, ..."


In [152]:
summary_buildings.head(2)

,id,ctuid,ctnum,geom
0,1,5350282.0,535028200.0,(POLYGON ((303785.8508000001 4841002.746400001...
1,2,5350265.0,535026500.0,"(POLYGON ((314413.1683999998 4842108.4136, 314..."


In [153]:
# writes blank minute columns _avg,_med,_min,_max to cts, summary_buildings and pednet_sum
out_columns = []
for measure in ['avg','med','min','max']:
    for current_column in buildings.columns:
        if current_column.startswith("m"):
            print(current_column)
            metric,poi,n = current_column.split("_")
            out_column = metric+"_"+poi+"_"+str(n)+"_"+measure
            out_columns.append(out_column)
            cts[out_column] = np.nan
            summary_buildings[out_column] = np.nan
            pednet_sum[out_column] = np.nan

m_fc_0
m_fc_1


/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


m_ff_0
m_ff_1
m_sm_0
m_sm_1
m_hp_0
m_hp_1
m_sxhc_0
m_sxhc_1
m_wc_0
m_wc_1
m_sc_0
m_sc_1
m_ar_0
m_ar_1
m_dc_0
m_dc_1
m_di_0
m_di_1
m_ems_0
m_ems_1
m_frc_0
m_frc_1
m_fr_0
m_fr_1
m_lb_0
m_lb_1
m_plc_0
m_plc_1
m_rec_0
m_rec_1
m_thing_0
m_thing_1
m_vtl_0
m_vtl_1
m_bp_0
m_bp_1
m_bs_0
m_bs_1
m_ttcstop_0
m_ttcstop_1
m_ttcst_0
m_ttcst_1
m_ttcacc_0
m_ttcacc_1
m_ts_0
m_ts_1
m_fc_0
m_fc_1
m_ff_0
m_ff_1
m_sm_0
m_sm_1
m_hp_0
m_hp_1
m_sxhc_0
m_sxhc_1
m_wc_0
m_wc_1
m_sc_0
m_sc_1
m_ar_0
m_ar_1
m_dc_0
m_dc_1
m_di_0
m_di_1
m_ems_0
m_ems_1
m_frc_0
m_frc_1
m_fr_0
m_fr_1
m_lb_0
m_lb_1
m_plc_0
m_plc_1
m_rec_0
m_rec_1
m_thing_0
m_thing_1
m_vtl_0
m_vtl_1
m_bp_0
m_bp_1
m_bs_0
m_bs_1
m_ttcstop_0
m_ttcstop_1
m_ttcst_0
m_ttcst_1
m_ttcacc_0
m_ttcacc_1
m_ts_0
m_ts_1
m_fc_0
m_fc_1
m_ff_0
m_ff_1
m_sm_0
m_sm_1
m_hp_0
m_hp_1
m_sxhc_0
m_sxhc_1
m_wc_0
m_wc_1
m_sc_0
m_sc_1
m_ar_0
m_ar_1
m_dc_0
m_dc_1
m_di_0
m_di_1
m_ems_0
m_ems_1
m_frc_0
m_frc_1
m_fr_0
m_fr_1
m_lb_0
m_lb_1
m_plc_0
m_plc_1
m_rec_0
m_rec_1
m_thing_0
m_thing_

In [154]:
buildings.head(2)

,id,geom,field_1,ctuid,ctname,ctnum,geotext,node_ids,d_fc_0,d_ff_0,d_sm_0,d_hp_0,d_sxhc_0,d_wc_0,d_sc_0,d_ar_0,d_dc_0,d_di_0,d_ems_0,d_fc_1,d_ff_1,d_sm_1,d_hp_1,d_sxhc_1,d_wc_1,d_sc_1,d_ar_1,d_dc_1,d_di_1,d_ems_1,d_frc_0,d_fr_0,d_lb_0,d_plc_0,d_rec_0,d_thing_0,d_vtl_0,d_bp_0,d_bs_0,d_ttcstop_0,d_ttcst_0,d_ttcacc_0,d_ts_0,d_frc_1,d_fr_1,d_lb_1,d_plc_1,d_rec_1,d_thing_1,d_vtl_1,d_bp_1,d_bs_1,d_ttcstop_1,d_ttcst_1,d_ttcacc_1,d_ts_1,m_fc_0,m_fc_1,m_ff_0,m_ff_1,m_sm_0,m_sm_1,m_hp_0,m_hp_1,m_sxhc_0,m_sxhc_1,m_wc_0,m_wc_1,m_sc_0,m_sc_1,m_ar_0,m_ar_1,m_dc_0,m_dc_1,m_di_0,m_di_1,m_ems_0,m_ems_1,m_frc_0,m_frc_1,m_fr_0,m_fr_1,m_lb_0,m_lb_1,m_plc_0,m_plc_1,m_rec_0,m_rec_1,m_thing_0,m_thing_1,m_vtl_0,m_vtl_1,m_bp_0,m_bp_1,m_bs_0,m_bs_1,m_ttcstop_0,m_ttcstop_1,m_ttcst_0,m_ttcst_1,m_ttcacc_0,m_ttcacc_1,m_ts_0,m_ts_1,n_fc_0n,n_fc_0type,n_ff_0n,n_ff_0type,n_sm_0n,n_sm_0type,n_hp_0n,n_hp_0type,n_sxhc_0n,n_sxhc_0type,n_wc_0n,n_wc_0type,n_sc_0n,n_sc_0type,n_ar_0n,n_ar_0type,n_lb_0n,n_lb_0type,n_dc_0n,n_dc_0type,n_fr_0n,n_fr_0type,n_di_0n,n_di_0type,n_ems_0n,n_ems_0type,n_frc_0n,n_frc_0type,n_plc_0n,n_plc_0type,n_rec_0n,n_rec_0type,n_thing_0n,n_thing_0type,n_vtl_0n,n_vtl_0type,n_bp_0type,n_bs_0type,n_ttcstop_0n,n_ttcstop_0type,n_ttcst_0n,n_ttcst_0type,n_ttcacc_0n,n_ttcacc_0type,n_ts_0n,n_ts_0type,n_fc_1n,n_fc_1type,n_ff_1n,n_ff_1type,n_sm_1n,n_sm_1type,n_hp_1n,n_hp_1type,n_sxhc_1n,n_sxhc_1type,n_wc_1n,n_wc_1type,n_sc_1n,n_sc_1type,n_ar_1n,n_ar_1type,n_lb_1n,n_lb_1type,n_dc_1n,n_dc_1type,n_fr_1n,n_fr_1type,n_di_1n,n_di_1type,n_ems_1n,n_ems_1type,n_frc_1n,n_frc_1type,n_plc_1n,n_plc_1type,n_rec_1n,n_rec_1type,n_thing_1n,n_thing_1type,n_vtl_1n,n_vtl_1type,n_bp_1type,n_bs_1type,n_ttcstop_1n,n_ttcstop_1type,n_ttcst_1n,n_ttcst_1type,n_ttcacc_1n,n_ttcacc_1type,n_ts_1n,n_ts_1type
0,1,(POLYGON ((303785.8508000001 4841002.746400001...,0,5350282.0,282.0,535028200.0,535028200,89416,487.143005,3179.489990,487.143005,3395.404053,2207.837891,2177.249023,60.861000,2361.337891,927.267029,1765.776001,1580.375977,570.526978,3179.489990,1656.931030,3606.419922,5000.0,3683.527100,696.325012,2678.096924,1544.262939,2385.887939,4537.008789,1624.584961,1580.375977,1708.963989,1535.342041,934.023987,3662.687988,696.325012,3376.153076,5000.0,175.798004,5000.000000,5000.000000,767.481018,1764.870972,2391.837891,2385.887939,1535.342041,1014.937012,4353.133789,850.755005,5000.000000,5000.0,198.593002,5000.000000,5000.000000,961.98999,6.765875,7.923986,44.159583,44.159583,6.765875,23.012931,47.158390,50.089166,30.664415,69.444444,30.239570,51.160099,0.845292,9.671181,32.796360,37.195791,12.878709,21.448096,24.524667,33.137332,21.949666,63.014011,22.563680,24.512097,21.949666,33.219971,23.735611,33.137332,21.324195,21.324195,12.972555,14.096347,50.870667,60.460192,9.671181,11.816042,46.891015,69.444444,69.444444,69.444444,23.735611,33.137332,69.444444,69.444444,69.444444,69.444444,10.659459,13.360972,Shop N Milk,cs,NEW PASSIONS RESTAURANT,ff,GRACE FOOD MARTS,sm,Trillium Health Centre - Queensway Health Centre,h,Birth Control and Sexual Health Centre,Eglinton-Lawrence (8),Downsview Medical Centre,w,TERRAVIEW WILLOWFIELD PUBLIC SCHOOL,EP,Cummer Park,North York,Weston Public Library,2 King St,Portage Trail B & A School Program,Portage Trail Junior Elementary School - Build...,FIRE STATION 441,947 MARTIN GROVE RD,Syme-Woolner Drop-In,Syme-Woolner Neighbourhood & Family Centre,EMS 19 Post,Ambulance Station 19,Falstaff Community Centre (OEYC),York South-Weston OEY - Macaulay Child Develop...,Marine Headquarters,259 Queens Quay West,Mount Dennis C. Hall,Recreation Centres,Humber Valley Golf Course,Golf Courses,Pelmo Park,Pellatt Ave,Bike Rack,None,GUILDWOOD PKWY AT PRINCE PHILLIP BLVD,2686,None,None,None,None,JANE ST,TRETHEWEY DR,Pelmo Mini Mart,cs,GUSTO PIZZA,ff,OYINGBO AFRICAN SUPERMARKET,sm,Centre For Addiction And Mental Health - Queen...,h,None,None,Patient Networks - Family Medicine and Health ...,w,PLEASANT PUBLIC SCHOOL,EP,Albion Arena,Etobicoke,None,1700 Wilson Ave,Galloway Ea

In [155]:
cts.head(2)

,gid,__gid,ctuid,ctname,pruid,prname,cmauid,cmapuid,cmaname,cmatype,shape_leng,shape_area,geotext,geom,m_fc_0_avg,m_fc_1_avg,m_ff_0_avg,m_ff_1_avg,m_sm_0_avg,m_sm_1_avg,m_hp_0_avg,m_hp_1_avg,m_sxhc_0_avg,m_sxhc_1_avg,m_wc_0_avg,m_wc_1_avg,m_sc_0_avg,m_sc_1_avg,m_ar_0_avg,m_ar_1_avg,m_dc_0_avg,m_dc_1_avg,m_di_0_avg,m_di_1_avg,m_ems_0_avg,m_ems_1_avg,m_frc_0_avg,m_frc_1_avg,m_fr_0_avg,m_fr_1_avg,m_lb_0_avg,m_lb_1_avg,m_plc_0_avg,m_plc_1_avg,m_rec_0_avg,m_rec_1_avg,m_thing_0_avg,m_thing_1_avg,m_vtl_0_avg,m_vtl_1_avg,m_bp_0_avg,m_bp_1_avg,m_bs_0_avg,m_bs_1_avg,m_ttcstop_0_avg,m_ttcstop_1_avg,m_ttcst_0_avg,m_ttcst_1_avg,m_ttcacc_0_avg,m_ttcacc_1_avg,m_ts_0_avg,m_ts_1_avg,m_fc_0_med,m_fc_1_med,m_ff_0_med,m_ff_1_med,m_sm_0_med,m_sm_1_med,m_hp_0_med,m_hp_1_med,m_sxhc_0_med,m_sxhc_1_med,m_wc_0_med,m_wc_1_med,m_sc_0_med,m_sc_1_med,m_ar_0_med,m_ar_1_med,m_dc_0_med,m_dc_1_med,m_di_0_med,m_di_1_med,m_ems_0_med,m_ems_1_med,m_frc_0_med,m_frc_1_med,m_fr_0_med,m_fr_1_med,m_lb_0_med,m_lb_1_med,m_plc_0_med,m_plc_1_med,m_rec_0_med,m_rec_1_med,m_thing_0_med,m_thing_1_med,m_vtl_0_med,m_vtl_1_med,m_bp_0_med,m_bp_1_med,m_bs_0_med,m_bs_1_med,m_ttcstop_0_med,m_ttcstop_1_med,m_ttcst_0_med,m_ttcst_1_med,m_ttcacc_0_med,m_ttcacc_1_med,m_ts_0_med,m_ts_1_med,m_fc_0_min,m_fc_1_min,m_ff_0_min,m_ff_1_min,m_sm_0_min,m_sm_1_min,m_hp_0_min,m_hp_1_min,m_sxhc_0_min,m_sxhc_1_min,m_wc_0_min,m_wc_1_min,m_sc_0_min,m_sc_1_min,m_ar_0_min,m_ar_1_min,m_dc_0_min,m_dc_1_min,m_di_0_min,m_di_1_min,m_ems_0_min,m_ems_1_min,m_frc_0_min,m_frc_1_min,m_fr_0_min,m_fr_1_min,m_lb_0_min,m_lb_1_min,m_plc_0_min,m_plc_1_min,m_rec_0_min,m_rec_1_min,m_thing_0_min,m_thing_1_min,m_vtl_0_min,m_vtl_1_min,m_bp_0_min,m_bp_1_min,m_bs_0_min,m_bs_1_min,m_ttcstop_0_min,m_ttcstop_1_min,m_ttcst_0_min,m_ttcst_1_min,m_ttcacc_0_min,m_ttcacc_1_min,m_ts_0_min,m_ts_1_min,m_fc_0_max,m_fc_1_max,m_ff_0_max,m_ff_1_max,m_sm_0_max,m_sm_1_max,m_hp_0_max,m_hp_1_max,m_sxhc_0_max,m_sxhc_1_max,m_wc_0_max,m_wc_1_max,m_sc_0_max,m_sc_1_max,m_ar_0_max,m_ar_1_max,m_dc_0_max,m_dc_1_max,m_di_0_max,m_di_1_max,m_ems_0_max,m_ems_1_max,m_frc_0_max,m_frc_1_max,m_fr_0_max,m_fr_1_max,m_lb_0_max,m_lb_1_max,m_plc_0_max,m_plc_1_max,m_rec_0_max,m_rec_1_max,m_thing_0_max,m_thing_1_max,m_vtl_0_max,m_vtl_1_max,m_bp_0_max,m_bp_1_max,m_bs_0_max,m_bs_1_max,m_ttcstop_0_max,m_ttcstop_1_max,m_ttcst_0_max,m_ttcst_1_max,m_ttcacc_0_max,m_ttcacc_1_max,m_ts_0_max,m_ts_1_max
ctnum,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
535000100.0,1,62,5350001.00,0001.00,35,Ontario,535,35535,Toronto,B,36906.495474,6.619690e+06,535000100,(POLYGON ((320588.9306980086 4835855.647789237...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
535000200.0,2,63,5350002.00,0002.00,35,Ontario,535,35535,Toronto,B,32534.307654,3.293422e+06,535000200,"(POLYGON ((316642.831678931 4832470.554474635,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [156]:
summary_buildings.head(2)

,id,ctuid,ctnum,geom,m_fc_0_avg,m_fc_1_avg,m_ff_0_avg,m_ff_1_avg,m_sm_0_avg,m_sm_1_avg,m_hp_0_avg,m_hp_1_avg,m_sxhc_0_avg,m_sxhc_1_avg,m_wc_0_avg,m_wc_1_avg,m_sc_0_avg,m_sc_1_avg,m_ar_0_avg,m_ar_1_avg,m_dc_0_avg,m_dc_1_avg,m_di_0_avg,m_di_1_avg,m_ems_0_avg,m_ems_1_avg,m_frc_0_avg,m_frc_1_avg,m_fr_0_avg,m_fr_1_avg,m_lb_0_avg,m_lb_1_avg,m_plc_0_avg,m_plc_1_avg,m_rec_0_avg,m_rec_1_avg,m_thing_0_avg,m_thing_1_avg,m_vtl_0_avg,m_vtl_1_avg,m_bp_0_avg,m_bp_1_avg,m_bs_0_avg,m_bs_1_avg,m_ttcstop_0_avg,m_ttcstop_1_avg,m_ttcst_0_avg,m_ttcst_1_avg,m_ttcacc_0_avg,m_ttcacc_1_avg,m_ts_0_avg,m_ts_1_avg,m_fc_0_med,m_fc_1_med,m_ff_0_med,m_ff_1_med,m_sm_0_med,m_sm_1_med,m_hp_0_med,m_hp_1_med,m_sxhc_0_med,m_sxhc_1_med,m_wc_0_med,m_wc_1_med,m_sc_0_med,m_sc_1_med,m_ar_0_med,m_ar_1_med,m_dc_0_med,m_dc_1_med,m_di_0_med,m_di_1_med,m_ems_0_med,m_ems_1_med,m_frc_0_med,m_frc_1_med,m_fr_0_med,m_fr_1_med,m_lb_0_med,m_lb_1_med,m_plc_0_med,m_plc_1_med,m_rec_0_med,m_rec_1_med,m_thing_0_med,m_thing_1_med,m_vtl_0_med,m_vtl_1_med,m_bp_0_med,m_bp_1_med,m_bs_0_med,m_bs_1_med,m_ttcstop_0_med,m_ttcstop_1_med,m_ttcst_0_med,m_ttcst_1_med,m_ttcacc_0_med,m_ttcacc_1_med,m_ts_0_med,m_ts_1_med,m_fc_0_min,m_fc_1_min,m_ff_0_min,m_ff_1_min,m_sm_0_min,m_sm_1_min,m_hp_0_min,m_hp_1_min,m_sxhc_0_min,m_sxhc_1_min,m_wc_0_min,m_wc_1_min,m_sc_0_min,m_sc_1_min,m_ar_0_min,m_ar_1_min,m_dc_0_min,m_dc_1_min,m_di_0_min,m_di_1_min,m_ems_0_min,m_ems_1_min,m_frc_0_min,m_frc_1_min,m_fr_0_min,m_fr_1_min,m_lb_0_min,m_lb_1_min,m_plc_0_min,m_plc_1_min,m_rec_0_min,m_rec_1_min,m_thing_0_min,m_thing_1_min,m_vtl_0_min,m_vtl_1_min,m_bp_0_min,m_bp_1_min,m_bs_0_min,m_bs_1_min,m_ttcstop_0_min,m_ttcstop_1_min,m_ttcst_0_min,m_ttcst_1_min,m_ttcacc_0_min,m_ttcacc_1_min,m_ts_0_min,m_ts_1_min,m_fc_0_max,m_fc_1_max,m_ff_0_max,m_ff_1_max,m_sm_0_max,m_sm_1_max,m_hp_0_max,m_hp_1_max,m_sxhc_0_max,m_sxhc_1_max,m_wc_0_max,m_wc_1_max,m_sc_0_max,m_sc_1_max,m_ar_0_max,m_ar_1_max,m_dc_0_max,m_dc_1_max,m_di_0_max,m_di_1_max,m_ems_0_max,m_ems_1_max,m_frc_0_max,m_frc_1_max,m_fr_0_max,m_fr_1_max,m_lb_0_max,m_lb_1_max,m_plc_0_max,m_plc_1_max,m_rec_0_max,m_rec_1_max,m_thing_0_max,m_thing_1_max,m_vtl_0_max,m_vtl_1_max,m_bp_0_max,m_bp_1_max,m_bs_0_max,m_bs_1_max,m_ttcstop_0_max,m_ttcstop_1_max,m_ttcst_0_max,m_ttcst_1_max,m_ttcacc_0_max,m_ttcacc_1_max,m_ts_0_max,m_ts_1_max
0,1,5350282.0,535028200.0,(POLYGON ((303785.8508000001 4841002.746400001...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,5350265.0,535026500.0,"(POLYGON ((314413.1683999998 4842108.4136, 314...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [157]:
summary_buildings.shape

(394475, 196)

In [158]:
# takes at least an hour to run?
start = time.time()
N = 572
c = 1
for group,df in buildings.groupby('ctnum'):
    #printing out summary for each census tract
    print("Doing CTNUM: ",group, "Number left: ", N-c)
    c+=1
    for current_column in df.columns:
        if current_column.startswith("m"):
            for measure in ['avg','med','min','max']:
                metric,poi,n = current_column.split("_")
                out_column = metric+"_"+poi+"_"+str(n)+"_"+measure
                res = {'avg':round(df[current_column].mean(),2),'med' : round(df[current_column].median(),2),
                       'min': round(df[current_column].min(),2),'max':round(df[current_column].max(),2)}
                #works for cts
                #cts.loc[group,out_column] =res[measure] 
                summary_buildings.loc[summary_buildings[summary_buildings.ctnum == group].index,out_column] =res[measure]
                #works for pednet
                #pednet_sum.loc[pednet_sum[pednet_sum.ctnum == group].index,out_column] =res[measure]
            #print("Finished Column(s) for: ", current_column)


Doing CTNUM:  535000100.0 Number left:  571
Doing CTNUM:  535000200.0 Number left:  570
Doing CTNUM:  535000300.0 Number left:  569
Doing CTNUM:  535000400.0 Number left:  568
Doing CTNUM:  535000500.0 Number left:  567
Doing CTNUM:  535000701.0 Number left:  566
Doing CTNUM:  535000702.0 Number left:  565
Doing CTNUM:  535000801.0 Number left:  564
Doing CTNUM:  535000802.0 Number left:  563
Doing CTNUM:  535000900.0 Number left:  562
Doing CTNUM:  535001001.0 Number left:  561
Doing CTNUM:  535001002.0 Number left:  560
Doing CTNUM:  535001100.0 Number left:  559
Doing CTNUM:  535001201.0 Number left:  558
Doing CTNUM:  535001203.0 Number left:  557
Doing CTNUM:  535001204.0 Number left:  556
Doing CTNUM:  535001301.0 Number left:  555
Doing CTNUM:  535001302.0 Number left:  554
Doing CTNUM:  535001400.0 Number left:  553
Doing CTNUM:  535001500.0 Number left:  552
Doing CTNUM:  535001600.0 Number left:  551
Doing CTNUM:  535001700.0 Number left:  550
Doing CTNUM:  535001800.0 Number

Doing CTNUM:  535016200.0 Number left:  384
Doing CTNUM:  535016300.0 Number left:  383
Doing CTNUM:  535016400.0 Number left:  382
Doing CTNUM:  535016500.0 Number left:  381
Doing CTNUM:  535016600.0 Number left:  380
Doing CTNUM:  535016701.0 Number left:  379
Doing CTNUM:  535016702.0 Number left:  378
Doing CTNUM:  535016800.0 Number left:  377
Doing CTNUM:  535016901.0 Number left:  376
Doing CTNUM:  535016902.0 Number left:  375
Doing CTNUM:  535017000.0 Number left:  374
Doing CTNUM:  535017100.0 Number left:  373
Doing CTNUM:  535017200.0 Number left:  372
Doing CTNUM:  535017300.0 Number left:  371
Doing CTNUM:  535017400.0 Number left:  370
Doing CTNUM:  535017501.0 Number left:  369
Doing CTNUM:  535017502.0 Number left:  368
Doing CTNUM:  535017600.0 Number left:  367
Doing CTNUM:  535018000.0 Number left:  366
Doing CTNUM:  535018101.0 Number left:  365
Doing CTNUM:  535018102.0 Number left:  364
Doing CTNUM:  535018200.0 Number left:  363
Doing CTNUM:  535018301.0 Number

Doing CTNUM:  535029902.0 Number left:  197
Doing CTNUM:  535030000.0 Number left:  196
Doing CTNUM:  535030101.0 Number left:  195
Doing CTNUM:  535030103.0 Number left:  194
Doing CTNUM:  535030104.0 Number left:  193
Doing CTNUM:  535030201.0 Number left:  192
Doing CTNUM:  535030202.0 Number left:  191
Doing CTNUM:  535030203.0 Number left:  190
Doing CTNUM:  535030300.0 Number left:  189
Doing CTNUM:  535030401.0 Number left:  188
Doing CTNUM:  535030402.0 Number left:  187
Doing CTNUM:  535030403.0 Number left:  186
Doing CTNUM:  535030404.0 Number left:  185
Doing CTNUM:  535030405.0 Number left:  184
Doing CTNUM:  535030406.0 Number left:  183
Doing CTNUM:  535030501.0 Number left:  182
Doing CTNUM:  535030503.0 Number left:  181
Doing CTNUM:  535030504.0 Number left:  180
Doing CTNUM:  535030601.0 Number left:  179
Doing CTNUM:  535030602.0 Number left:  178
Doing CTNUM:  535030703.0 Number left:  177
Doing CTNUM:  535030704.0 Number left:  176
Doing CTNUM:  535030705.0 Number

Doing CTNUM:  535037823.0 Number left:  8
Doing CTNUM:  535037824.0 Number left:  7
Doing CTNUM:  535037825.0 Number left:  6
Doing CTNUM:  535037826.0 Number left:  5
Doing CTNUM:  535037827.0 Number left:  4
Doing CTNUM:  535037828.0 Number left:  3
Doing CTNUM:  535080201.0 Number left:  2
Doing CTNUM:  535080202.0 Number left:  1


In [159]:
summary_buildings[summary_buildings.ctnum == group].index

Int64Index([169412, 169416, 169420, 169424, 169428, 169432, 169436, 169440,
            169444, 169448,
            ...
            173786, 173787, 173788, 173789, 173790, 173791, 173792, 173793,
            173794, 173795],
           dtype='int64', length=1644)

In [160]:
#summary_buildings doesn't have the ctnum column
summary_buildings.dtypes

id                   int64
ctuid              float64
ctnum              float64
geom                object
m_fc_0_avg         float64
m_fc_1_avg         float64
m_ff_0_avg         float64
m_ff_1_avg         float64
m_sm_0_avg         float64
m_sm_1_avg         float64
m_hp_0_avg         float64
m_hp_1_avg         float64
m_sxhc_0_avg       float64
m_sxhc_1_avg       float64
m_wc_0_avg         float64
m_wc_1_avg         float64
m_sc_0_avg         float64
m_sc_1_avg         float64
m_ar_0_avg         float64
m_ar_1_avg         float64
m_dc_0_avg         float64
m_dc_1_avg         float64
m_di_0_avg         float64
m_di_1_avg         float64
m_ems_0_avg        float64
m_ems_1_avg        float64
m_frc_0_avg        float64
m_frc_1_avg        float64
m_fr_0_avg         float64
m_fr_1_avg         float64
m_lb_0_avg         float64
m_lb_1_avg         float64
m_plc_0_avg        float64
m_plc_1_avg        float64
m_rec_0_avg        float64
m_rec_1_avg        float64
m_thing_0_avg      float64
m

In [161]:
cts.head(2)

,gid,__gid,ctuid,ctname,pruid,prname,cmauid,cmapuid,cmaname,cmatype,shape_leng,shape_area,geotext,geom,m_fc_0_avg,m_fc_1_avg,m_ff_0_avg,m_ff_1_avg,m_sm_0_avg,m_sm_1_avg,m_hp_0_avg,m_hp_1_avg,m_sxhc_0_avg,m_sxhc_1_avg,m_wc_0_avg,m_wc_1_avg,m_sc_0_avg,m_sc_1_avg,m_ar_0_avg,m_ar_1_avg,m_dc_0_avg,m_dc_1_avg,m_di_0_avg,m_di_1_avg,m_ems_0_avg,m_ems_1_avg,m_frc_0_avg,m_frc_1_avg,m_fr_0_avg,m_fr_1_avg,m_lb_0_avg,m_lb_1_avg,m_plc_0_avg,m_plc_1_avg,m_rec_0_avg,m_rec_1_avg,m_thing_0_avg,m_thing_1_avg,m_vtl_0_avg,m_vtl_1_avg,m_bp_0_avg,m_bp_1_avg,m_bs_0_avg,m_bs_1_avg,m_ttcstop_0_avg,m_ttcstop_1_avg,m_ttcst_0_avg,m_ttcst_1_avg,m_ttcacc_0_avg,m_ttcacc_1_avg,m_ts_0_avg,m_ts_1_avg,m_fc_0_med,m_fc_1_med,m_ff_0_med,m_ff_1_med,m_sm_0_med,m_sm_1_med,m_hp_0_med,m_hp_1_med,m_sxhc_0_med,m_sxhc_1_med,m_wc_0_med,m_wc_1_med,m_sc_0_med,m_sc_1_med,m_ar_0_med,m_ar_1_med,m_dc_0_med,m_dc_1_med,m_di_0_med,m_di_1_med,m_ems_0_med,m_ems_1_med,m_frc_0_med,m_frc_1_med,m_fr_0_med,m_fr_1_med,m_lb_0_med,m_lb_1_med,m_plc_0_med,m_plc_1_med,m_rec_0_med,m_rec_1_med,m_thing_0_med,m_thing_1_med,m_vtl_0_med,m_vtl_1_med,m_bp_0_med,m_bp_1_med,m_bs_0_med,m_bs_1_med,m_ttcstop_0_med,m_ttcstop_1_med,m_ttcst_0_med,m_ttcst_1_med,m_ttcacc_0_med,m_ttcacc_1_med,m_ts_0_med,m_ts_1_med,m_fc_0_min,m_fc_1_min,m_ff_0_min,m_ff_1_min,m_sm_0_min,m_sm_1_min,m_hp_0_min,m_hp_1_min,m_sxhc_0_min,m_sxhc_1_min,m_wc_0_min,m_wc_1_min,m_sc_0_min,m_sc_1_min,m_ar_0_min,m_ar_1_min,m_dc_0_min,m_dc_1_min,m_di_0_min,m_di_1_min,m_ems_0_min,m_ems_1_min,m_frc_0_min,m_frc_1_min,m_fr_0_min,m_fr_1_min,m_lb_0_min,m_lb_1_min,m_plc_0_min,m_plc_1_min,m_rec_0_min,m_rec_1_min,m_thing_0_min,m_thing_1_min,m_vtl_0_min,m_vtl_1_min,m_bp_0_min,m_bp_1_min,m_bs_0_min,m_bs_1_min,m_ttcstop_0_min,m_ttcstop_1_min,m_ttcst_0_min,m_ttcst_1_min,m_ttcacc_0_min,m_ttcacc_1_min,m_ts_0_min,m_ts_1_min,m_fc_0_max,m_fc_1_max,m_ff_0_max,m_ff_1_max,m_sm_0_max,m_sm_1_max,m_hp_0_max,m_hp_1_max,m_sxhc_0_max,m_sxhc_1_max,m_wc_0_max,m_wc_1_max,m_sc_0_max,m_sc_1_max,m_ar_0_max,m_ar_1_max,m_dc_0_max,m_dc_1_max,m_di_0_max,m_di_1_max,m_ems_0_max,m_ems_1_max,m_frc_0_max,m_frc_1_max,m_fr_0_max,m_fr_1_max,m_lb_0_max,m_lb_1_max,m_plc_0_max,m_plc_1_max,m_rec_0_max,m_rec_1_max,m_thing_0_max,m_thing_1_max,m_vtl_0_max,m_vtl_1_max,m_bp_0_max,m_bp_1_max,m_bs_0_max,m_bs_1_max,m_ttcstop_0_max,m_ttcstop_1_max,m_ttcst_0_max,m_ttcst_1_max,m_ttcacc_0_max,m_ttcacc_1_max,m_ts_0_max,m_ts_1_max
ctnum,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
535000100.0,1,62,5350001.00,0001.00,35,Ontario,535,35535,Toronto,B,36906.495474,6.619690e+06,535000100,(POLYGON ((320588.9306980086 4835855.647789237...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
535000200.0,2,63,5350002.00,0002.00,35,Ontario,535,35535,Toronto,B,32534.307654,3.293422e+06,535000200,"(POLYGON ((316642.831678931 4832470.554474635,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [162]:
pednet_sum.head(2)

,gid_left,ctuid,ctnum,geom,m_fc_0_avg,m_fc_1_avg,m_ff_0_avg,m_ff_1_avg,m_sm_0_avg,m_sm_1_avg,m_hp_0_avg,m_hp_1_avg,m_sxhc_0_avg,m_sxhc_1_avg,m_wc_0_avg,m_wc_1_avg,m_sc_0_avg,m_sc_1_avg,m_ar_0_avg,m_ar_1_avg,m_dc_0_avg,m_dc_1_avg,m_di_0_avg,m_di_1_avg,m_ems_0_avg,m_ems_1_avg,m_frc_0_avg,m_frc_1_avg,m_fr_0_avg,m_fr_1_avg,m_lb_0_avg,m_lb_1_avg,m_plc_0_avg,m_plc_1_avg,m_rec_0_avg,m_rec_1_avg,m_thing_0_avg,m_thing_1_avg,m_vtl_0_avg,m_vtl_1_avg,m_bp_0_avg,m_bp_1_avg,m_bs_0_avg,m_bs_1_avg,m_ttcstop_0_avg,m_ttcstop_1_avg,m_ttcst_0_avg,m_ttcst_1_avg,m_ttcacc_0_avg,m_ttcacc_1_avg,m_ts_0_avg,m_ts_1_avg,m_fc_0_med,m_fc_1_med,m_ff_0_med,m_ff_1_med,m_sm_0_med,m_sm_1_med,m_hp_0_med,m_hp_1_med,m_sxhc_0_med,m_sxhc_1_med,m_wc_0_med,m_wc_1_med,m_sc_0_med,m_sc_1_med,m_ar_0_med,m_ar_1_med,m_dc_0_med,m_dc_1_med,m_di_0_med,m_di_1_med,m_ems_0_med,m_ems_1_med,m_frc_0_med,m_frc_1_med,m_fr_0_med,m_fr_1_med,m_lb_0_med,m_lb_1_med,m_plc_0_med,m_plc_1_med,m_rec_0_med,m_rec_1_med,m_thing_0_med,m_thing_1_med,m_vtl_0_med,m_vtl_1_med,m_bp_0_med,m_bp_1_med,m_bs_0_med,m_bs_1_med,m_ttcstop_0_med,m_ttcstop_1_med,m_ttcst_0_med,m_ttcst_1_med,m_ttcacc_0_med,m_ttcacc_1_med,m_ts_0_med,m_ts_1_med,m_fc_0_min,m_fc_1_min,m_ff_0_min,m_ff_1_min,m_sm_0_min,m_sm_1_min,m_hp_0_min,m_hp_1_min,m_sxhc_0_min,m_sxhc_1_min,m_wc_0_min,m_wc_1_min,m_sc_0_min,m_sc_1_min,m_ar_0_min,m_ar_1_min,m_dc_0_min,m_dc_1_min,m_di_0_min,m_di_1_min,m_ems_0_min,m_ems_1_min,m_frc_0_min,m_frc_1_min,m_fr_0_min,m_fr_1_min,m_lb_0_min,m_lb_1_min,m_plc_0_min,m_plc_1_min,m_rec_0_min,m_rec_1_min,m_thing_0_min,m_thing_1_min,m_vtl_0_min,m_vtl_1_min,m_bp_0_min,m_bp_1_min,m_bs_0_min,m_bs_1_min,m_ttcstop_0_min,m_ttcstop_1_min,m_ttcst_0_min,m_ttcst_1_min,m_ttcacc_0_min,m_ttcacc_1_min,m_ts_0_min,m_ts_1_min,m_fc_0_max,m_fc_1_max,m_ff_0_max,m_ff_1_max,m_sm_0_max,m_sm_1_max,m_hp_0_max,m_hp_1_max,m_sxhc_0_max,m_sxhc_1_max,m_wc_0_max,m_wc_1_max,m_sc_0_max,m_sc_1_max,m_ar_0_max,m_ar_1_max,m_dc_0_max,m_dc_1_max,m_di_0_max,m_di_1_max,m_ems_0_max,m_ems_1_max,m_frc_0_max,m_frc_1_max,m_fr_0_max,m_fr_1_max,m_lb_0_max,m_lb_1_max,m_plc_0_max,m_plc_1_max,m_rec_0_max,m_rec_1_max,m_thing_0_max,m_thing_1_max,m_vtl_0_max,m_vtl_1_max,m_bp_0_max,m_bp_1_max,m_bs_0_max,m_bs_1_max,m_ttcstop_0_max,m_ttcstop_1_max,m_ttcst_0_max,m_ttcst_1_max,m_ttcacc_0_max,m_ttcacc_1_max,m_ts_0_max,m_ts_1_max
0,34488,5350013.02,535001302.0,"(LINESTRING (314815.8607 4833592.2401, 314821....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,22847,5350013.02,535001302.0,"(LINESTRING (314792.3111 4833576.0342, 314797....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [ ]:
#You always need to drop the geometry column and write a new WKT text geometry column 

cts1 = cts.copy()
cts1['geometry'] = cts1['geom']
cts1['geom'] = cts1['geometry'].apply(lambda x: WKTElement(x.wkt, srid=2019))
#drop the geometry column as it is now duplicative
cts1.drop('geometry', 1, inplace=True)
#Adds objectid as double. Need to change to integer
cts1['objectid'] = cts1.index.astype(np.int64)
cts1.to_sql('census_summary_walktimes_fin', walkability_engine, schema='results', if_exists='replace', index=True, 
                         dtype={'geom': Geometry('MULTIPOLYGON', srid=2019)})

In [ ]:
#keeping only valid polygon geometries
summary_buildings_out = summary_buildings[summary_buildings.is_valid]

In [ ]:
#You always need to drop the geometry column and write a new WKT text geometry column
summary_buildings1 = summary_buildings_out.copy()
summary_buildings1['geometry'] = summary_buildings1['geom']
summary_buildings1['geom'] = summary_buildings1['geometry'].apply(lambda x: WKTElement(x.wkt, srid=2019))
#drop the geometry column as it is now duplicative
summary_buildings1.drop('geometry', 1, inplace=True)
#Adds objectid as double.
#summary_buildings1['objectid'] = summary_buildings1.index.astype(np.int64)
#summary_buildings1['objectid'] = list(range(1,summary_buildings1.shape[0]))
summary_buildings1.to_sql('buildings_summary_walktimes_fin',walkability_engine, schema='results', if_exists='replace', index=True, 
                         dtype={'geom': Geometry('MULTIPOLYGON', srid=2019)})

In [ ]:

#You always need to drop the geometry column and write a new WKT text geometry column 
pednet_sum1 = pednet_sum.copy()
pednet_sum1['geometry'] = pednet_sum1['geom']
pednet_sum1['geom'] = pednet_sum1['geometry'].apply(lambda x: WKTElement(x.wkt, srid=2019))
#drop the geometry column as it is now duplicative
pednet_sum1.drop('geometry', 1, inplace=True)
#Adds objectid as double.
#objectid = 9306 is duplicate
pednet_sum1['objectid'] = list(range(1,pednet_sum1.shape[0]))
pednet_sum1.to_sql('pednet_summary_walktimes_fin', walkability_engine, schema='results', if_exists='replace', index=True, 
                         dtype={'geom': Geometry('MULTILINESTRING', srid=2019)})